### CÉLULA DE INSTALAÇÃO (IMPORTANTE)

Se você está recebendo o erro `ModuleNotFoundError` (ex: `No module named 'seaborn'`), execute a célula de código abaixo **UMA ÚNICA VEZ**.

Depois que ela terminar de instalar, **REINICIE O KERNEL** (no menu superior: Kernel > Restart) e só então execute o restante do notebook a partir da "Célula 1".

In [36]:
# --- CÉLULA DE EMERGÊNCIA PARA INSTALAÇÃO ---
# Execute esta célula UMA VEZ se estiver faltando alguma biblioteca.
# Depois, REINICIE O KERNEL antes de continuar.

!pip install jupyter pandas numpy matplotlib seaborn scikit-learn xgboost joblib imbalanced-learn


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Luke\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Tarefa 2: Classificação Binária de Empréstimos

*(Nota: Este é o arquivo `tarefa_2_mnist.ipynb` corrigido para a tarefa de Empréstimos)*

Este notebook segue os passos da avaliação:
a) Separar treino e teste (usando `train_test_split` com `stratify`)
b) Remover colunas solicitadas.
c) Verificar e preencher dados faltantes (Imputação).
d) Confirmar dados faltantes.
e) LabelEncoder nas features dicotômicas.
f) Tratar a feature 'Dependents'.
g) Padronizar (StandardScaler) features numéricas.
h) Criar 5 Pipelines (com SMOTE e GridSearchCV).
i) Escolher e salvar o melhor pipeline.

### Célula 1: Importação das Bibliotecas

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# SKLearn - Divisão e Métricas
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# SKLearn - Pré-processamento
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# SKLearn - Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Imbalanced-Learn (para SMOTE)
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

# Configurações
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

### Célula 2: Etapa 1 - Carregar Dados

Carregamos o arquivo `loan.csv`.

In [38]:
try:
    df = pd.read_csv('loan.csv')
    print("Arquivo 'loan.csv' carregado com sucesso.")
except FileNotFoundError:
    print("Erro: Arquivo 'loan.csv' não encontrado.")
    print("Por favor, certifique-se de que o arquivo 'loan.csv' está no mesmo diretório que este notebook.")

Arquivo 'loan.csv' carregado com sucesso.


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Loan_ID            215 non-null    object
 1   Gender             215 non-null    object
 2   Married            215 non-null    object
 3   Dependents         215 non-null    object
 4   Education          215 non-null    object
 5   Self_Employed      212 non-null    object
 6   ApplicantIncome    215 non-null    object
 7   CoapplicantIncome  215 non-null    object
 8   LoanAmount         214 non-null    object
 9   Loan_Amount_Term   214 non-null    object
 10  Credit_History     211 non-null    object
 11  Property_Area      215 non-null    object
 12  Loan_Status        213 non-null    object
dtypes: object(13)
memory usage: 22.0+ KB


In [40]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0,NaN,360,1,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508,128,360,1,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0,66,360,1,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358,120,360,1,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0,141,360,1,Urban,Y


### Célula 3: Etapa 2a - Divisão de Treino e Teste (COM CORREÇÃO)

Aqui estava o erro. Aplicamos `dropna(subset=['Loan_Status'])` **antes** de dividir X e y, para permitir que o `stratify=y` funcione.

In [41]:
# 1. (Etapa 2b) Remover a coluna 'Loan_ID', pois é apenas um identificador
if 'Loan_ID' in df.columns:
    df = df.drop(columns=['Loan_ID'])

# --- INÍCIO DA CORREÇÃO ---
# 2. Remover linhas onde o *alvo* (Loan_Status) é nulo.
# Isso é essencial para o stratify=y funcionar.
print(f"Linhas antes de limpar NaNs do 'Loan_Status': {len(df)}")
df = df.dropna(subset=['Loan_Status'])
print(f"Linhas após limpar NaNs do 'Loan_Status': {len(df)}")
# --- FIM DA CORREÇÃO ---

# 3. Definir X e y (Agora com 'y' limpo)
y = df['Loan_Status']
X = df.drop(columns='Loan_Status')

# 4. Separar Treino e Teste (Etapa 2a)
# Esta linha (antiga linha 18 do traceback) agora funciona.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nFormato Treino: {X_train.shape}")
print(f"Formato Teste: {X_test.shape}")

Linhas antes de limpar NaNs do 'Loan_Status': 215
Linhas após limpar NaNs do 'Loan_Status': 213

Formato Treino: (170, 11)
Formato Teste: (43, 11)


### Célula 4: Mapear o Alvo (y)

O Scikit-learn espera que o alvo `y` seja numérico (0 ou 1). Vamos mapear 'N' para 0 e 'Y' para 1.

In [42]:
# Mapear y_train e y_test
y_train_encoded = y_train.map({'N': 0, 'Y': 1})
y_test_encoded = y_test.map({'N': 0, 'Y': 1})

print("Valores únicos em y (original):", y_train.unique())
print("Valores únicos em y (codificado):", y_train_encoded.unique())

# Verificar balanceamento (importante para o SMOTE)
print("\nBalanceamento de y_train_encoded:")
print(y_train_encoded.value_counts(normalize=True))

Valores únicos em y (original): ['Y' 'N']
Valores únicos em y (codificado): [1 0]

Balanceamento de y_train_encoded:
Loan_Status
1    0.847059
0    0.152941
Name: proportion, dtype: float64


### Célula 5: Etapa 2 (c, e, f, g) - Criar Pré-processadores

Vamos criar os transformadores que serão usados no `ColumnTransformer`.

In [43]:
# 1. (Etapa 2f) Tratar 'Dependents': substituir '3+' por '3' e preencher NaNs com '0' (estratégia mais comum)
X_train['Dependents'] = X_train['Dependents'].replace('3+', '3')
X_test['Dependents'] = X_test['Dependents'].replace('3+', '3')

# 2. Definir os grupos de colunas (baseado no df.info())

# Colunas numéricas que precisam de Imputação (mediana) e Scaler
numeric_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']

# Colunas categóricas que precisam de Imputação (moda) e OneHotEncoder
# (Etapa 2e/f - O pipeline trata 'Dependents' e 'Gender'/'Married'/'Self_Employed')
categorical_features = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']

# Coluna 'Credit_History' é tratada separadamente, pois já é quase binária (0.0, 1.0, NaN)
# Vamos apenas imputar a moda (1.0) e não aplicar scaler.
binary_feature = ['Credit_History']

# 3. Criar os mini-pipelines de transformação

# (Etapa 2c) Imputação + (Etapa 2g) Scaler
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# (Etapa 2c) Imputação + (Etapa 2e) OneHotEncoder
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# (Etapa 2c) Imputação para a coluna binária
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

# 4. Combinar tudo no ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bin', binary_transformer, binary_feature)
    ],
    remainder='passthrough' # Mantém colunas não especificadas (embora não haja nenhuma aqui)
)

print("Pré-processador criado com sucesso.")

Pré-processador criado com sucesso.


### Célula 6: Etapa 2h - Definir os 5 Pipelines de Modelo

Criamos 5 pipelines completos, cada um contendo:
1. `preprocessor`: O ColumnTransformer que criamos.
2. `smote`: O SMOTE para balancear os dados (somente no treino).
3. `model`: O classificador.

In [44]:
# Modelos a serem testados
models = {
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'SVC': SVC(probability=True, random_state=42),
    'RandomForestClassifier': RandomForestClassifier(random_state=42),
    'XGBClassifier': XGBClassifier(random_state=42, eval_metric='logloss', use_label_encoder=False)
}

# Parâmetros para o GridSearchCV (exemplos simples)
# (Para uma busca real, seriam mais extensos)
param_grids = {
    'LogisticRegression': {
        'model__C': [0.1, 1.0, 10]
    },
    'KNeighborsClassifier': {
        'model__n_neighbors': [3, 5, 7]
    },
    'SVC': {
        'model__C': [0.1, 1.0],
        'model__kernel': ['linear', 'rbf']
    },
    'RandomForestClassifier': {
        'model__n_estimators': [50, 100],
        'model__max_depth': [5, 10]
    },
    'XGBClassifier': {
        'model__n_estimators': [50, 100],
        'model__learning_rate': [0.01, 0.1]
    }
}

pipelines = {}

for model_name, model in models.items():
    pipelines[model_name] = ImbPipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('model', model)
    ])
    
print(f"{len(pipelines)} pipelines criados (com SMOTE).")

5 pipelines criados (com SMOTE).


### Célula 7: Etapa 2h (continuação) - Treinar com GridSearchCV

Iteramos pelos 5 pipelines e treinamos cada um usando GridSearchCV para encontrar os melhores hiperparâmetros.

In [45]:
cv_results = {}
best_estimators = {}
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for model_name in pipelines.keys():
    print(f"--- Treinando {model_name} ---")
    pipe = pipelines[model_name]
    param_grid = param_grids[model_name]
    
    # Usamos o y_train_encoded (0/1)
    grid_search = GridSearchCV(pipe, param_grid, cv=kfold, scoring='accuracy', n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train_encoded)
    
    cv_results[model_name] = {
        'best_score': grid_search.best_score_,
        'best_params': grid_search.best_params_
    }
    
    best_estimators[model_name] = grid_search.best_estimator_
    
    print(f"Melhor Acurácia (CV): {grid_search.best_score_:.4f}")
    print(f"Melhores Parâmetros: {grid_search.best_params_}")
    print("--------------------------------\n")

--- Treinando LogisticRegression ---
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Melhor Acurácia (CV): 0.8647
Melhores Parâmetros: {'model__C': 1.0}
--------------------------------

--- Treinando KNeighborsClassifier ---
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Melhor Acurácia (CV): 0.7059
Melhores Parâmetros: {'model__n_neighbors': 3}
--------------------------------

--- Treinando SVC ---
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Melhor Acurácia (CV): 0.8412
Melhores Parâmetros: {'model__C': 1.0, 'model__kernel': 'linear'}
--------------------------------

--- Treinando RandomForestClassifier ---
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Melhor Acurácia (CV): 0.8941
Melhores Parâmetros: {'model__max_depth': 5, 'model__n_estimators': 50}
--------------------------------

--- Treinando XGBClassifier ---
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Melhor Acurácia (CV): 0.8824
Melhores Parâmetros: {'m

### Célula 8: Etapa 2i - Avaliar, Escolher e Salvar o Melhor Modelo

Avaliamos o desempenho no conjunto de TESTE e salvamos o melhor pipeline.

In [46]:
test_scores = {}
best_pipeline = None
best_test_score = 0.0
best_model_name = ""

print("--- Avaliação no Conjunto de Teste ---")

for model_name, estimator in best_estimators.items():
    y_pred = estimator.predict(X_test)
    
    # Usamos o y_test_encoded (0/1) para comparar
    test_accuracy = accuracy_score(y_test_encoded, y_pred)
    test_scores[model_name] = test_accuracy
    
    print(f"Modelo: {model_name}")
    print(f"  Acurácia (Teste): {test_accuracy:.4f}")
    print(f"  Acurácia (CV): {cv_results[model_name]['best_score']:.4f}")
    print(classification_report(y_test_encoded, y_pred, target_names=['N', 'Y']))
    
    # (Etapa 2i) Escolher o melhor
    if test_accuracy > best_test_score:
        best_test_score = test_accuracy
        best_pipeline = estimator
        best_model_name = model_name

print(f"\n--- Melhor Modelo Escolhido (Etapa 2i) ---")
print(f"Modelo: {best_model_name}")
print(f"Acurácia (Teste): {best_test_score:.4f}")

# Salvar o melhor pipeline (Etapa 2j)
pipeline_filename = 'loan_pipeline.joblib'
if best_pipeline:
    joblib.dump(best_pipeline, pipeline_filename)
    print(f"\nMelhor pipeline salvo como: {pipeline_filename}")
else:
    print("\nNenhum modelo foi treinado para salvar.")

--- Avaliação no Conjunto de Teste ---
Modelo: LogisticRegression
  Acurácia (Teste): 0.8372
  Acurácia (CV): 0.8647
              precision    recall  f1-score   support

           N       0.50      0.71      0.59         7
           Y       0.94      0.86      0.90        36

    accuracy                           0.84        43
   macro avg       0.72      0.79      0.74        43
weighted avg       0.87      0.84      0.85        43

Modelo: KNeighborsClassifier
  Acurácia (Teste): 0.6977
  Acurácia (CV): 0.7059
              precision    recall  f1-score   support

           N       0.20      0.29      0.24         7
           Y       0.85      0.78      0.81        36

    accuracy                           0.70        43
   macro avg       0.52      0.53      0.52        43
weighted avg       0.74      0.70      0.72        43

Modelo: SVC
  Acurácia (Teste): 0.8605
  Acurácia (CV): 0.8412
              precision    recall  f1-score   support

           N       0.56      0.

### Célula 9: Etapa 2d e 2j - Verificações Finais

Confirmamos que não há NaNs após o pré-processamento e testamos o carregamento do pipeline salvo.

In [47]:
if best_pipeline:
    # Etapa 2d: Confirmar se não há dados faltantes (após o pipeline)
    # (Acessamos o 'preprocessor' dentro do pipeline do imblearn)
    X_train_processed = best_pipeline.named_steps['preprocessor'].transform(X_train)
    
    # --- CORREÇÃO (2ª TENTATIVA) ---
    # O erro 'AttributeError' mostrou que X_train_processed JÁ É um numpy.ndarray (denso).
    # O erro original 'TypeError' ocorria porque o array tem dtype=object.
    # A solução é converter para float ANTES de usar np.isnan().
    nan_count = np.isnan(X_train_processed.astype(float)).sum()
    # --- FIM DA CORREÇÃO ---
    
    print(f"\nVerificação (Etapa 2d): Dados faltantes após o pré-processamento: {nan_count}")
    
    # Testar o carregamento e predição do pipeline salvo (Etapa 2j - preparação)
    print("\nVerificando o pipeline salvo...")
    try:
        loaded_pipe = joblib.load(pipeline_filename)
        
        # Pegar a primeira amostra do X_test (antes de qualquer tratamento manual)
        sample = X_test.iloc[[0]]
        
        prediction = loaded_pipe.predict(sample)
        probability = loaded_pipe.predict_proba(sample)
        
        print(f"Amostra de teste (entrada):\n {sample}")
        print(f"Predição (0=N, 1=Y): {prediction[0]}")
        print(f"Probabilidades (N, Y): {probability[0]}")
        print(f"Predição Real (0=N, 1=Y): {y_test_encoded.iloc[0]}")
    except FileNotFoundError:
        print("Arquivo 'loan_pipeline.joblib' não encontrado para teste.")
else:
    print("\nVerificação final pulada pois nenhum modelo foi treinado.")


Verificação (Etapa 2d): Dados faltantes após o pré-processamento: 0

Verificando o pipeline salvo...
Amostra de teste (entrada):
     Gender Married Dependents Education Self_Employed ApplicantIncome  \
130   Male      No          0  Graduate            No           13237   

    CoapplicantIncome LoanAmount Loan_Amount_Term Credit_History Property_Area  
130                 0        160              360              1     Semiurban  
Predição (0=N, 1=Y): 1
Probabilidades (N, Y): [0.20570135 0.79429865]
Predição Real (0=N, 1=Y): 1
